In [3]:
from battery.battery import Battery
from price_modelling.price_path_generator import IDPriceSimulator
from optimization.rolling_intrinsic import RollingIntrinsicStrategy
import pandas as pd
import yaml
import numpy as np

Tune ID price simulator to data in ../data

In [4]:
df_all_VWAP_data = pd.read_csv("../data/all_VWAP_DA_data_spring.csv", sep=";")
# set columns to datetime type
df_all_VWAP_data["delivery_start"] = pd.to_datetime(df_all_VWAP_data["delivery_start"])
df_all_VWAP_data["delivery_start_hour"] = pd.to_datetime(df_all_VWAP_data["delivery_start_hour"])
df_all_VWAP_data["traded"] = pd.to_datetime(df_all_VWAP_data["traded"])

price_generator = IDPriceSimulator()
price_generator.fit_model(df_all_VWAP_data)

/Users/rickvanvoorbergen/Mijn Drive/Studie/Masters/Thesis/battery_intraday_trading/src/price_modelling/price_path_generator.py:106: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/rickvanvoorbergen/Mijn Drive/Studie/Masters/Thesis/battery_intraday_trading/src/price_modelling/price_path_generator.py:106: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/rickvanvoorbergen/Mijn Drive/Studie/Masters/Thesis/battery_intraday_trading/src/price_modelling/price_path_generator.py:106: SettingWithCopyWarning:


Setup Battery

In [5]:
# read config.yml
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)  

battery = Battery(config)

Create test price path array

In [6]:
price_path_array, P = price_generator.generate_price_paths(season="spring", N_paths=1)

In [5]:
# price_path_array = np.ones((1,config["N_trading_timesteps"],config["N_timesteps"]))

# price_path_array[0,0,1:4] = 5
# price_path_array[0,0,26:29] = 0

# price_path_array[0,1,4:7] = -10
# price_path_array[0,1,23:26] = 2

# price_path_array[0,4,10:15] = -30
# # price_path_array[0,4,26:29] = 0

# # price_path_array[0,4,4:7] = -10
# # price_path_array[0,4,23:26] = 2





In [7]:
intrinsic_strategy = RollingIntrinsicStrategy(battery, price_path_array, config)

In [8]:
power_schedules, soc_schedules, price_path, results = intrinsic_strategy.run()

we are here
Solver did not find an optimal solution at timestep 58


In [8]:
eps = 1e-12

# Test if schedules are within bounds
for power_schedule in power_schedules:
    assert np.all(np.abs(power_schedule) <= battery.capacity)

for soc_schedule in soc_schedules:
    assert np.all(soc_schedule >= -eps) and np.all(soc_schedule <= 1+eps)

# Test if results are equal to power_schedule multiplied by price_path
for t in range(2, len(results) - 1):
    # print(t)
    assert np.abs(results[t]-results[t-1] - np.sum((power_schedules[t] - power_schedules[t-1]) * price_path[0,t,:])) < eps

# Test if the number of cycles is less than the maximum allowed
# assert np.sum(np.abs(power_schedules[-1]))/2 * battery.RTE <= battery.N_daily_cycles_max
# # Lower bound on revenues
# costs = np.sum(power_schedules * price_path[0,:,:], axis=1)
# # assert np.all(costs >= 0)
# costs

In [15]:
np.sum(np.abs(power_schedules[56]))/2 * battery.time_step

np.float64(0.9791666666666666)

In [13]:
# plot schedule 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

timestep = 57
# Add SOC schedule trace on primary y-axis
fig.add_trace(
    go.Scatter(
        x=list(range(len(soc_schedules[0]))),
        y=soc_schedules[timestep],
        name="SOC",
        line=dict(color="#00BFFF"),  # Bright blue for better visibility on dark theme
        mode="lines",
        line_shape="hv"
    ),
    secondary_y=False
)

# Add price path trace on secondary y-axis
fig.add_trace(
    go.Scatter(
        x=list(range(len(price_path[0,timestep,:]))),
        y=price_path[0,timestep,:],
        name="Price",
        line=dict(color="#FF5733"),  # Bright orange-red for better visibility
        mode="lines",
        line_shape="hv"
    ),
    secondary_y=True
)

# Set titles and labels with dark theme
fig.update_layout(
    title_text="Battery State of Charge and Price Path",
    hovermode="x unified",
    template="plotly_dark",  # Use dark template
    paper_bgcolor="rgb(30, 30, 30)",  # Dark background color
    plot_bgcolor="rgb(30, 30, 30)",
    font=dict(color="white")  # White text
)
fig.update_xaxes(title_text="Time Step", gridcolor="rgba(255, 255, 255, 0.2)")
fig.update_yaxes(title_text="State of Charge (SOC)", secondary_y=False, gridcolor="rgba(255, 255, 255, 0.2)")
fig.update_yaxes(title_text="Price", secondary_y=True, gridcolor="rgba(255, 255, 255, 0.2)")

# Show the interactive plot
fig.show()

In [12]:
results

array([ 456.41708201,  496.47212854,  496.54798879,  496.54798879,
        496.54798879,  496.54798879,  496.78063093,  497.40885984,
        497.40885984,  498.07869692,  498.17040324,  498.17040324,
        498.17040324,  498.2541443 ,  498.83411952,  498.83411952,
        499.75122706,  501.30859163,  501.30859163,  501.30859163,
        501.30859163,  501.30859163,  501.30859163,  501.30859163,
        501.30859163,  501.84516053,  501.84516053,  502.62027295,
        503.25984394,  503.25984394,  503.25984394,  503.25984394,
        503.25984394,  503.25984394,  503.25984394,  503.38296179,
        503.38296179,  503.59039654,  503.59039654,  503.59039654,
        504.0624972 ,  504.0624972 ,  577.88201345,  578.54344068,
        660.58306905,  660.58306905,  662.19609351,  673.65191768,
        673.65191768,  673.65191768,  677.07611363,  680.91365991,
        680.91365991,  773.15805127,  863.51411741,  959.40809894,
       1039.64494654, 1222.92082094, 1222.92082094, 1233.41633